# 0. Notebook Settings

In [1]:
SW_KEY_PATH = "/kaggle/input/serviceworker/football-151224-034eb3fe0abb.json"
SW_FOLDER_ID = "1bN4QJZIdJsdftA4xcO2YbVOgE20mQ1YA"

PROJ_DIR = "/kaggle/working/"
# urls of EPL matches from the last 5 seasons
IDlist = {
    '2018-2019' : range(1284741, 1285121),
    '2019-2020' : range(1375927, 1376307),
    '2020-2021' : range(1485184, 1485564),
    '2021-2022' : range(1549539, 1549919),
    '2022-2023' : range(1640674, 1641054),
    '2023-2024' : range(1729191, 1729569)
}
# IDlist = {
#     'test': [1284741, 1284742, 1284743, 1284744, 1284745, 1284746]
# }

NUM_THREADS = 4

In [2]:
!lscpu | grep -vE "Vulnerability"

Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Byte Order:                           Little Endian
Address sizes:                        46 bits physical, 48 bits virtual
CPU(s):                               4
On-line CPU(s) list:                  0-3
Thread(s) per core:                   2
Core(s) per socket:                   2
Socket(s):                            1
NUMA node(s):                         1
Vendor ID:                            GenuineIntel
CPU family:                           6
Model:                                79
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                             0
CPU MHz:                              2199.998
BogoMIPS:                             4399.99
Hypervisor vendor:                    KVM
Virtualization type:                  full
L1d cache:                            64 KiB
L1i cache:                            64 KiB
L2 cache:                  

# 1. Setting Up the Environment

## 1.1. Install dependencies:
This command updates the system's package list and installs various libraries required for running Chrome and Selenium.

In [3]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libdbus-glib-1-2 \
libgconf-2-4 \
libfontconfig1 \
libvulkan1 \
gconf2-common \
libwayland-server0 \
libgbm1 \
udev \
libu2f-udev 
!apt --fix-broken install -y  

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]    
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]      
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1588 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4596 kB]
Get:9 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3446 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [34.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1568 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4348 kB]
Get:13 http://security.ubunt

## 1.2. Download and extract Chrome:

To use Selenium, you will need to download and install Chrome and Chromedriver.

* **Chrome**: Chrome is a popular web browser that is known for its speed and security.
* **Chromedriver**: Chromedriver is a tool that allows Selenium to interact with Chrome.

Downloads the latest stable version of Chrome for Linux and extracts it to the /usr/bin directory.

In [4]:
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/

--2024-12-15 14:49:38--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip [following]
--2024-12-15 14:49:38--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.20.207, 74.125.142.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145898081 (139M) [application/zip]
Saving to: '/tmp/chrome-linux64.zip'

chrome-linux64.zip  100%[===================>] 139.14M   212MB/s    i

## 1.3. Download and extract Chromedriver:

As it was done in the previous code.

In [5]:
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/

--2024-12-15 14:49:45--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip [following]
--2024-12-15 14:49:45--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 108.177.98.207, 74.125.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7271942 (6.9M) [application/zip]
Saving to: '/tmp/chromedriver-linux64.zip'

chromedriver-linux6 100%[===================>]

## 1.4. Install Python libraries

In [6]:
!apt install -y python3-selenium
!pip install selenium==3.141.0
!pip install pydrive2==1.21.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 143 not upgraded.
Need to get 26.9 MB of archives.
After this operation, 113 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.4 [505 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd am

# 2. Configuring Chrome Driver

These functions define the locations of Chrome and Chromedriver executables. Additionally, initialize_driver creates a Chrome webdriver instance with specific options:

* *--headless*: Runs Chrome in headless mode, making it invisible.
* *--no-sandbox*: Disables the sandbox for improved performance.
* *--start-fullscreen*: Starts Chrome in fullscreen mode.
* *--allow-insecure-localhost*: Allows access to insecure local websites (if needed).
* *--disable-dev-shm-usage*: Disables shared memory usage for Chrome.
* *user-agent*: Sets the user agent string to mimic a regular browser.

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import json
import requests
import csv
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta
import os
from retrying import retry
import time
import traceback
from tqdm import tqdm
from pathlib import Path
import concurrent.futures
import threading

CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=options)
    return driver


# 3. Scrape with Selenium

In [8]:
!rm -r /kaggle/working/

rm: cannot remove '/kaggle/working/': Device or resource busy


In [9]:
pregame_data_folder = os.path.join(PROJ_DIR, 'data', 'pregame_data')
Path(pregame_data_folder).mkdir(parents=True, exist_ok=True)

event_data_folder = os.path.join(PROJ_DIR, 'data', 'event_data')
Path(event_data_folder).mkdir(parents=True, exist_ok=True)

In [10]:
# path fo pregame data
csv_file = os.path.join(PROJ_DIR, 'data', 'pregame_data', 'pregame_data.csv')

# attributes for pregame data
keylist = ['match_id', 'date', 'home_team', 'away_team', 'home_team_id', 'away_team_id', 'home_team_elo', 'away_team_elo']

# Define a lock for thread-safe file writing
lock = threading.Lock()

def scrape_match(matchID, writer):
    matchID = str(matchID)
    url = 'https://1xbet.whoscored.com/Matches/' + matchID + '/Live'

    # Open web driver on the url
    driver = initialize_driver()
    driver.get(url)

    # Get the JSON data
    data_xpath = '/html/body/div[4]/script[1]'
    wait = WebDriverWait(driver, 300)
    data_element = wait.until(EC.presence_of_element_located((By.XPATH, data_xpath)))
    data = data_element.get_attribute('innerHTML')
    start_index = data.find('matchCentreData')
    end_index = data.find('matchCentreEventTypeJson')

    data = data[start_index + 17:end_index - 14]
    data = json.loads(data)

    # Extract event data
    event_data = data['events']
    df = pd.DataFrame(event_data)
    df.to_csv(os.path.join(PROJ_DIR, 'data', 'event_data', matchID + '.csv'), index=False)

    # Get the match date to crawl ELO rating data
    date = data['startTime'][:-9]
    ddate = datetime.strptime(date, '%Y-%m-%d')

    previous_day = ddate - timedelta(days=1)
    predate = previous_day.strftime('%Y-%m-%d')

    # Get data from clubelo api
    r = requests.get('http://api.clubelo.com/' + predate)
    elo_data = StringIO(r.text)
    elo = pd.read_csv(elo_data, sep=",")

    # Get team name
    name_xpath = '/html/body/div[4]/div[3]/div[1]/div[2]/table/tbody/tr[1]'
    wait = WebDriverWait(driver, 300)
    name_element = wait.until(EC.presence_of_element_located((By.XPATH, name_xpath)))
    name = name_element.get_attribute('innerHTML')
    driver.quit()

    end_index = name.find('</a>')
    start_index = name[:end_index].rfind('>')
    home_team_name = name[start_index + 1:end_index]

    # Some team names need to be synchronized from Whoscored to clubelo
    if home_team_name == 'Man Utd': home_team_name = 'Man United'
    if home_team_name == 'WBA': home_team_name = 'West Brom'
    if home_team_name == 'Sheff Utd': home_team_name = 'Sheffield United'
    if home_team_name == 'Nottingham Forest': home_team_name = 'Forest'
    end_index = name.rfind('</a>')
    start_index = name[:end_index].rfind('>')
    away_team_name = name[start_index + 1:end_index]
    if away_team_name == 'Man Utd': away_team_name = 'Man United'
    if away_team_name == 'WBA': away_team_name = 'West Brom'
    if away_team_name == 'Sheff Utd': away_team_name = 'Sheffield United'
    if away_team_name == 'Nottingham Forest': away_team_name = 'Forest'

    # Get elo ratings data
    home_team_elo = elo.loc[elo['Club'] == home_team_name, 'Elo'].values[0]
    away_team_elo = elo.loc[elo['Club'] == away_team_name, 'Elo'].values[0]

    # Store pregame data
    instance_data = {
        'match_id': matchID,
        'date': date,
        'home_team': home_team_name,
        'away_team': away_team_name,
        'home_team_id': data['home']['teamId'],
        'away_team_id': data['away']['teamId'],
        'home_team_elo': home_team_elo,
        'away_team_elo': away_team_elo
    }

    # Write to CSV file in a thread-safe manner
    with lock:
        writer.writerow(instance_data)

# Main function to run the scraping with multithreading
def main():
    with open(csv_file, 'a+', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=keylist)
        if file.tell() == 0:
            writer.writeheader()

        with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
            futures = []
            for season, urls in IDlist.items():
                for matchID in urls:
                    futures.append(executor.submit(scrape_match, matchID, writer))

                for _ in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scraping season {season}"):
                    pass

print("Done.")

Done.


In [11]:
main()

Scraping season 2023-2024: 100%|██████████| 2278/2278 [15:19<00:00,  2.48it/s] 


# 4. Data integration

In [15]:
import os
import pandas as pd
import json
import ast

In [16]:
match_data_folder = os.path.join(PROJ_DIR, 'data', 'match')
Path(match_data_folder).mkdir(parents=True, exist_ok=True)

In [20]:
#get the path to pregame data
pregame_data = pd.read_csv(os.path.join(PROJ_DIR, 'data', 'pregame_data', 'pregame_data.csv'))

In [21]:
for i in pregame_data.index:
    #default data instance
    game_data = [{
        'minute' : 0,
        'half' : 1,
        'ht_elo': pregame_data['home_team_elo'][i],
        'at_elo' : pregame_data['away_team_elo'][i],
        'ht_goal' : 0,
        'at_goal' : 0,
        'pass' : 0,
        'short_pass' : 0,
        'long_pass' : 0,
        'final_3rd_pass' : 0,
        'key_pass' : 0,
        'cross' : 0,
        'corner' : 0,
        'big_chance' : 0,
        'shot' : 0,
        'shot_6_yard_box' : 0,
        'shot_penalty_box' : 0,
        'shot_open_play' : 0,
        'shot_fast_break' : 0,
        'dispossessed' : 0,
        'turnover' : 0,
        'duel' : 0,
        'tackle' : 0,
        'interception' : 0,
        'clearance' : 0,
        'offside' : 0,
        'yellow' : 0,
        'red' : 0,
        'result' : 'D'
    }]
    #home team and away team id
    ht_id = pregame_data['home_team_id'][i]
    at_id = pregame_data['away_team_id'][i]

    #get the event data of the corresponding pregame data instance from the 'match_id' attrinbute
    event_data = pd.read_csv(os.path.join(PROJ_DIR, 'data', 'event_data', str(pregame_data['match_id'][i])+'.csv'))

    #transform data into dictionary type
    event_data['period'] = event_data['period'].apply(ast.literal_eval)
    event_data['type'] = event_data['type'].apply(ast.literal_eval)
    event_data['satisfiedEventsTypes'] = event_data['satisfiedEventsTypes'].apply(ast.literal_eval)

    #iterate through event to calculate values of needed attributes
    new_event = game_data[-1].copy()
    for ind, event in event_data.iterrows():
        new_event['minute'] = event['minute']
        new_event['half'] = event['period']['value']
        
        #only create new data instance whenever the minute change
        if new_event['minute'] > game_data[-1]['minute'] or (new_event['half'] > game_data[-1]['half'] and new_event['half'] == 2):
            game_data.append(new_event.copy())

        #using dictionary from event['type']['value'] to determine event type
        if event['type']['value'] == 16:
            if 23 in event['satisfiedEventsTypes']:
                if event['teamId'] == ht_id:
                    new_event['at_goal'] += 1
                else:
                    new_event['ht_goal'] += 1
            else:
                if event['teamId'] == ht_id:
                    new_event['ht_goal'] += 1
                else:
                    new_event['at_goal'] += 1
        if 117 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['pass'] += 1
            else:
                new_event['pass'] -= 1
        if 30 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['short_pass'] += 1
            else:
                new_event['short_pass'] -= 1
        if 127 in event['satisfiedEventsTypes'] or 128 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['long_pass'] += 1
            else:
                new_event['long_pass'] -= 1
        if 217 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['final_3rd_pass'] += 1
            else:
                new_event['final_3rd_pass'] -= 1
        if 123 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['key_pass'] += 1
            else:
                new_event['key_pass'] -= 1
        if 125 in event['satisfiedEventsTypes'] or 126 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['cross'] += 1
            else:
                new_event['cross'] -= 1
        if 31 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['corner'] += 1
            else:
                new_event['corner'] -= 1
        if 203 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['big_chance'] += 1
            else:
                new_event['big_chance'] -= 1
        if 10 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['shot'] += 1
            else:
                new_event['shot'] -= 1
        if 0 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['shot_6_yard_box'] += 1
            else:
                new_event['shot_6_yard_box'] -= 1
        if 1 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['shot_penalty_box'] += 1
            else:
                new_event['shot_penalty_box'] -= 1
        if 3 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['shot_open_play'] += 1
            else:
                new_event['shot_open_play'] -= 1
        if 4 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['shot_fast_break'] += 1
            else:
                new_event['shot_fast_break'] -= 1
        if 70 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['dispossessed'] += 1
            else:
                new_event['dispossessed'] -= 1
        if 69 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['turnover'] += 1
            else:
                new_event['turnover'] -= 1
        if 197 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['duel'] += 1
            else:
                new_event['duel'] -= 1
        if 143 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['tackle'] += 1
            else:
                new_event['tackle'] -= 1
        if 101 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['interception'] += 1
            else:
                new_event['interception'] -= 1
        if 95 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['clearance'] += 1
            else:
                new_event['clearance'] -= 1
        if 61 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['offside'] += 1
            else:
                new_event['offside'] -= 1
        if 65 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['yellow'] += 1
            else:
                new_event['yellow'] -= 1
        if 68 in event['satisfiedEventsTypes']:
            if event['teamId'] == ht_id:
                new_event['red'] += 1
            else:
                new_event['red'] -= 1
    
    df = pd.DataFrame(game_data)

    #result attribute from home team goal and away team goal
    diff = game_data[-1]['ht_goal'] - game_data[-1]['at_goal']    
    if diff > 0:
        df['result'] = 'W'
    elif diff < 0:
        df['result'] = 'L'

    #store data in csv file
    df.to_csv(os.path.join(PROJ_DIR, 'data', 'match', str(pregame_data['match_id'][i])+'.csv'), index=False)
    if not os.path.isfile(os.path.join(PROJ_DIR, 'data', 'full.csv')):
        df.to_csv(os.path.join(PROJ_DIR, 'data', 'full.csv'), mode='a', index=False, header=df.keys)
    else:        
        df.to_csv(os.path.join(PROJ_DIR, 'data', 'full.csv'), mode='a', index=False, header=False)

# 5. Train Models

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt

In [27]:
data = pd.read_csv(os.path.join(PROJ_DIR, 'data', 'full.csv'))
data.head()

,minute,half,ht_elo,at_elo,ht_goal,at_goal,pass,short_pass,long_pass,final_3rd_pass,...,dispossessed,turnover,duel,tackle,interception,clearance,offside,yellow,red,result
0,0,1,1567.101318,1837.004272,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,L
1,1,1,1567.101318,1837.004272,0,0,-7,-6,0,-1,...,0,-1,0,0,1,1,0,0,0,L
2,2,1,1567.101318,1837.004272,0,0,-10,-9,1,-2,...,0,-1,0,1,1,1,0,0,0,L
3,3,1,1567.101318,1837.004272,0,0,-10,-9,3,-6,...,0,-1,0,1,0,1,0,0,0,L
4,4,1,1567.101318,1837.004272,0,0,-13,-13,4,-8,...,0,-2,0,1,-1,1,-1,0,0,L


In [25]:
data[(data['minute']) == 0 & (data['half'] == 1)].shape

(12, 29)

In [28]:
# Map 'result' values to numerical values
mapping = {'W': 2, 'D': 1, 'L': 0}
data['result_numerical'] = data['result'].map(mapping)

data.drop('result', axis=1, inplace=True)
data.rename(columns={'result_numerical': 'result'}, inplace=True)

data.head()

,minute,half,ht_elo,at_elo,ht_goal,at_goal,pass,short_pass,long_pass,final_3rd_pass,...,dispossessed,turnover,duel,tackle,interception,clearance,offside,yellow,red,result
0,0,1,1567.101318,1837.004272,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1567.101318,1837.004272,0,0,-7,-6,0,-1,...,0,-1,0,0,1,1,0,0,0,0
2,2,1,1567.101318,1837.004272,0,0,-10,-9,1,-2,...,0,-1,0,1,1,1,0,0,0,0
3,3,1,1567.101318,1837.004272,0,0,-10,-9,3,-6,...,0,-1,0,1,0,1,0,0,0,0
4,4,1,1567.101318,1837.004272,0,0,-13,-13,4,-8,...,0,-2,0,1,-1,1,-1,0,0,0


In [31]:
# Filter data for the first half
first_half_data = data[data['half'] == 1]

# Select features (exclude 'result' and other irrelevant columns)
features = ['minute', 'home_goals', 'away_goals']  # Add other relevant features
X = first_half_data.drop('result', axis=1)
y = first_half_data['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Custom Ranked Logarithmic Score function
def ranked_logarithmic_score(y_true, y_pred_prob):
    """
    Calculate Ranked Logarithmic Score (RLS).
    y_true: Ground truth labels (1D array).
    y_pred_prob: Predicted probability distributions (2D array).
    """
    y_true_onehot = np.zeros_like(y_pred_prob)
    y_true_onehot[np.arange(len(y_true)), y_true] = 1  # One-hot encode y_true

    cumulative_true = np.cumsum(y_true_onehot, axis=1)
    cumulative_pred = np.cumsum(y_pred_prob, axis=1)

    rls = np.mean(np.sum((cumulative_true - cumulative_pred) ** 2, axis=1))
    return rls

# Custom scorer for GridSearchCV
def rls_scorer(estimator, X, y):
    y_pred_prob = estimator.predict_proba(X)
    return -ranked_logarithmic_score(y, y_pred_prob)  # Negative for minimization

rls_scorer_fn = make_scorer(rls_scorer, greater_is_better=False, needs_proba=True)

In [ ]:
param_grid = {
    'n_estimators': [30, 50],
    'max_depth': [7],
}

model = XGBClassifier(use_label_encoder=False, objective="multi:softprob", eval_metric='mlogloss')

# Split data for validation during training
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Use GridSearchCV with custom scorer
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=rls_scorer_fn, n_jobs=-1)
grid_search.fit(X_train_part, y_train_part)

# Retrieve the best model
best_model = grid_search.best_estimator_

# Predict probabilities on the training, validation, and test sets
y_train_pred_prob = best_model.predict_proba(X_train)
y_val_pred_prob = best_model.predict_proba(X_val)
y_test_pred_prob = best_model.predict_proba(X_test)

# 6. Update models to Drive

In [ ]:
from pydrive2.fs import GDriveFileSystem

fs = GDriveFileSystem(
    # replace with ID of a drive or directory and give service account access to it
    SW_FOLDER_ID,
    use_service_account=True,
    client_json_file_path=SW_KEY_PATH,
)

In [ ]:
# replace `root` with ID of a drive or directory and give service account access to it
for root, dnames, fnames in fs.walk(SW_FOLDER_ID):
    for dname in dnames:
        print(f"dir: {root}/{dname}")

    for fname in fnames:
        print(f"file: {root}/{fname}")